# Augmented line connections

This notebook will compare the original, clustered and augmented line network.
To reproduce the images, run the workflow with the augmented line connection.

```python
augmented_line_connection:
  add_to_snakefile: true  # If True, includes this rule to the workflow
```

## Parameters and imports

### Python imports

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point
from shapely.geometry import LineString

import os
import sys

sys.path.append("../")  # to import helpers
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")
sys.path.append("./scripts")  # to import helpers
from download_osm_data import create_country_list

### Paths

The below paths are required to generate the images. We use wildcards from glob to access multiple paths. '*' is interpreted as any value for any number of characters. '?' is interpreted as any value for only one character.


In [ ]:
import glob

# network paths
path_networks = glob.glob("networks/elec_s_*nc")
clustered_network = path_networks[0]
simplified_network = path_networks[1]
pre_augmented_network = path_networks[2]
augmented_network = path_networks[3]
path_networks

# Example output
# ['networks/elec_s_420.nc',
# 'networks/elec_s_420_ec.nc',
# 'networks/elec_s_420_pre_augmentation.nc',
# 'networks/elec_s_420_ec_lcopt_Co2L-24.nc']

In [ ]:
# Paths of build shapes
path_country_shapes = "resources/country_shapes.geojson"
path_offshore_shapes = "resources/offshore_shapes.geojson"
path_gadm_shapes = "resources/gadm_shapes.geojson"
path_cluster_shape_on = glob.glob("resources/regions_onshore_elec_s_*.geojson")[0]
path_cluster_shape_off = glob.glob("resources/regions_offshore_elec_s_*.geojson")[0]

# Path of the OSM data
path_raw_substations = "data/raw/africa_all_raw_substations.geojson"
path_raw_lines = "data/raw/africa_all_raw_lines.geojson"

In [ ]:
# Images setups
max_width_image = 15
max_height_image = 15
dpi = 300

### Auxiliary functions

In [ ]:
def calculate_width_height_image(
    width_image, height_image, max_width=max_width_image, max_height=max_height_image
):
    """
    Function to identify the width and height of an image to plot
    while keeping the proportions of the image
    """
    if width_image / height_image >= max_width / max_height:
        # image width is the limiting factor
        return (width_image, max_width / width_image * height_image)
    else:
        # image height is the limiting factor
        return (max_height / height_image * width_image, max_height)

### File imports

In [ ]:
# gadm file imports
countries = gpd.read_file(path_country_shapes)
off_shore = gpd.read_file(path_offshore_shapes)
gadm = gpd.read_file(path_gadm_shapes)
cluster_shape_on = gpd.read_file(path_cluster_shape_on)
cluster_shape_off = gpd.read_file(path_cluster_shape_off)

# OSM data imports
df_substations_osm_raw = gpd.read_file(path_raw_substations)
df_lines_osm_raw = gpd.read_file(path_raw_lines)

# network imports
n_cluster = pypsa.Network(clustered_network)
n_simple = pypsa.Network(simplified_network)
n_preaugmented = pypsa.Network(pre_augmented_network)
n_augmented = pypsa.Network(augmented_network)

## Continent and raw OSM data

### Plot for the entire area downloaded

In [ ]:
import contextily as cx  # Need to be installed `pip3 install contextily`
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib as mpl

total_bounds_countries = countries.total_bounds
delta_bounds_xy = (
    total_bounds_countries[2] - total_bounds_countries[0],  # maxx - minx
    total_bounds_countries[3] - total_bounds_countries[1],
)  # maxy - miny

size_image = calculate_width_height_image(*delta_bounds_xy)

ax2 = gadm.plot(
    column="pop",
    cmap="OrRd",
    figsize=size_image,
    legend=False,
    norm=matplotlib.colors.LogNorm(
        vmin=gadm["pop"].min() + 1, vmax=gadm["pop"].max(), clip=True
    ),
)
# ax2 = cluster_shape_on.plot(figsize=size_image, legend=None)
cluster_shape_off.plot(ax=ax2, label="offshore")
df_lines_osm_raw.plot(ax=ax2, color="navy", label="OSM lines")
df_substations_osm_raw.plot(
    ax=ax2, color="papayawhip", alpha=0.7, markersize=5, label="OSM substations"
)


# Colorbar
cmap = "OrRd"
norm = matplotlib.colors.LogNorm(
    vmin=gadm["pop"].min() / 1000 + 1, vmax=gadm["pop"].max() / 1000, clip=True
)
cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax2,
    orientation="vertical",
    extend="max",
    shrink=0.6,
)
cbar.set_label("Population/1000 in administrative zones", fontsize=16, color="dimgrey")


plt.axis("off")

plt.rc("legend", fontsize=13)
ax2.legend(
    bbox_to_anchor=(1.035, 0.95),
    borderaxespad=0,
    facecolor="#1f77b4",
    framealpha=0.9,
    labelcolor="white",
)
plt.savefig("build-osm-network-africa.pdf", bbox_inches="tight")

### Plot for a subregion specified by a string as in "countries" config

In [ ]:
region = ["NG"]

list_countries = create_country_list(region)

ax2 = gadm[gadm.country.isin(list_countries)].plot(
    column="pop",
    cmap="OrRd",
    figsize=size_image,
    legend=None,
    norm=matplotlib.colors.LogNorm(
        vmin=gadm["pop"].min() + 1, vmax=gadm["pop"].max(), clip=True
    ),
)  # column="pop",
off_shore[off_shore.name.isin(list_countries)].plot(ax=ax2, label="offshore")
df_lines_osm_raw[df_lines_osm_raw.Country.isin(list_countries)].plot(
    ax=ax2, color="navy"
)
df_substations_osm_raw[df_substations_osm_raw.Country.isin(list_countries)].plot(
    ax=ax2, color="papayawhip", alpha=0.7, markersize=15
)

# Colorbar
cmap = "OrRd"
norm = matplotlib.colors.LogNorm(
    vmin=gadm["pop"].min() / 1000 + 1, vmax=gadm["pop"].max() / 1000, clip=True
)
cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax2,
    orientation="vertical",
    extend="max",
    shrink=0.6,
)
cbar.set_label("Population/1000 in administrative zones", fontsize=16, color="dimgrey")

plt.axis("off")

## Cluster network

In [ ]:
### EXTRACT DATA FIRST OF PYPSA NETWORK

# Options
# clustered_network = path_networks[0]
# simplified_network = path_networks[1]
# pre_augmented_network = path_networks[2]
# augmented_network = path_networks[3]

# buses dataframe
buses_c = n_cluster.buses
buses_c["geometry"] = gpd.points_from_xy(buses_c.x, buses_c.y)
buses_c = gpd.GeoDataFrame(buses_c, crs="epsg:4326")

# lines dataframe
lines_c = n_cluster.lines
lines_c["geometry"] = lines_c.apply(
    lambda x: LineString(
        [buses_c.loc[x["bus0"], "geometry"], buses_c.loc[x["bus1"], "geometry"]]
    ),
    axis=1,
)
lines_c = gpd.GeoDataFrame(lines_c, crs="epsg:4326")


### Plot cluster network

In [ ]:
region = ["Africa"]
list_countries = create_country_list(region)

ax2 = gadm[gadm.country.isin(list_countries)].plot(
    column="pop",
    cmap="OrRd",
    figsize=size_image,
    legend=None,
    norm=matplotlib.colors.LogNorm(
        vmin=gadm["pop"].min() + 1, vmax=gadm["pop"].max(), clip=True
    ),
)  # column="pop",
off_shore[off_shore.name.isin(list_countries)].plot(ax=ax2, label="offshore")


lines_c.plot(ax=ax2, color="navy", label="Clustered lines")
buses_c.plot(
    ax=ax2, color="papayawhip", alpha=0.9, markersize=15, label="Clustered substations"
)


# Colorbar
cmap = "OrRd"
norm = matplotlib.colors.LogNorm(
    vmin=gadm["pop"].min() / 1000 + 1, vmax=gadm["pop"].max() / 1000, clip=True
)
cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax2,
    orientation="vertical",
    extend="max",
    shrink=0.6,
)
cbar.set_label("Population/1000 in administrative zones", fontsize=16, color="dimgrey")

plt.axis("off")
plt.rc("legend", fontsize=13)
ax2.legend(
    bbox_to_anchor=(1.035, 0.95),
    borderaxespad=0,
    facecolor="#1f77b4",
    framealpha=0.9,
    labelcolor="white",
)
plt.savefig("420-clustered-network-africa.pdf", bbox_inches="tight")

## Augmented network

In [ ]:
### Creates a LineString of the Augmented lines

# buses dataframe
buses_c = n_augmented.buses
buses_c["geometry"] = gpd.points_from_xy(buses_c.x, buses_c.y)
buses_c = gpd.GeoDataFrame(buses_c, crs="epsg:4326")

# lines dataframe
lines_c = n_augmented.lines
lines_c["geometry"] = lines_c.apply(
    lambda x: LineString(
        [buses_c.loc[x["bus0"], "geometry"], buses_c.loc[x["bus1"], "geometry"]]
    ),
    axis=1,
)
lines_c = gpd.GeoDataFrame(lines_c, crs="epsg:4326")

# links dataframe
links_c = n_augmented.links
links_c["geometry"] = np.nan
for i in range(len(n_augmented.links.bus0)):
    n_augmented.links.iloc[i, -1] = LineString(
        [
            n_augmented.buses.loc[
                n_augmented.buses.index == n_augmented.links.bus0[i], "geometry"
            ][0],
            n_augmented.buses.loc[
                n_augmented.buses.index == n_augmented.links.bus1[i], "geometry"
            ][0],
        ]
    )
links_c = gpd.GeoDataFrame(n_augmented.links, crs="epsg:4326")

In [ ]:
region = ["Africa"]
list_countries = create_country_list(region)

ax2 = gadm[gadm.country.isin(list_countries)].plot(
    column="pop",
    cmap="OrRd",
    figsize=size_image,
    legend=None,
    norm=matplotlib.colors.LogNorm(
        vmin=gadm["pop"].min() + 1, vmax=gadm["pop"].max(), clip=True
    ),
)  # column="pop",
off_shore[off_shore.name.isin(list_countries)].plot(ax=ax2, label="offshore")

lines_c.plot(ax=ax2, color="navy", label="Clustered lines")
links_c.plot(
    ax=ax2, color="springgreen", linewidth=5, alpha=0.7, label="Augmented lines"
)
buses_c.plot(
    ax=ax2, color="papayawhip", alpha=1, markersize=15, label="Clustered substations"
)

# Colorbar
cmap = "OrRd"
norm = matplotlib.colors.LogNorm(
    vmin=gadm["pop"].min() / 1000 + 1, vmax=gadm["pop"].max() / 1000, clip=True
)
cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax2,
    orientation="vertical",
    extend="max",
    shrink=0.6,
)
cbar.set_label("Population/1000 in administrative zones", fontsize=16, color="dimgrey")

plt.axis("off")

plt.rc("legend", fontsize=13)
ax2.legend(
    bbox_to_anchor=(0.75, 0.95),
    borderaxespad=0,
    facecolor="#1f77b4",
    framealpha=0.9,
    labelcolor="white",
)
plt.savefig("420-augmented-line-africa.pdf", bbox_inches="tight")